In [41]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer


In [42]:

#faults_orig = pd.read_csv('../data/sp500.csv', low_memory=False, parse_dates=['EventTimeStamp', 'LocationTimeStamp']) #1187335
sp_all = pd.read_csv('../data/sp500.csv', low_memory=False,parse_dates=['Date'])
inflation = pd.read_csv("../data/InflationRates.csv")
cpi = pd.read_csv("../data/cpi_alpha_cleaned-final.csv")
intRate = pd.read_csv("../data/IntRate_Fed_Daily.csv")
unemployment = pd.read_csv("../data/unemployment_bls.csv")
npayroll = pd.read_csv("../data/non_farm_payroll_bls.csv")
vix = pd.read_csv("../data/vix.csv")


In [ ]:
vix

In [ ]:
npayroll

In [ ]:
# sp_all
#inflation
# cpi
#intRate
unemployment

In [23]:
unemployment

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2019,4.0,3.8,3.8,3.6,3.7,3.6,3.7,3.7,3.5,3.6,3.6,3.6
72,2020,3.5,3.5,4.4,14.7,13.2,11.0,10.2,8.4,7.9,6.9,6.7,6.7
73,2021,6.3,6.2,6.1,6.1,5.8,5.9,5.4,5.2,4.8,4.5,4.2,3.9
74,2022,4.0,3.8,3.6,3.6,3.6,3.6,3.5,3.7,3.5,3.7,3.6,3.5


In [ ]:
nan_count = sp_all.isna().sum() 
nan_count

# Data Cleanup

## Convert Date column to datetime and strip the time part

In [45]:
sp_all['Date'] = pd.to_datetime(sp_all['Date'], utc=True).dt.date
sp_all['Date'] = sp_all['Date'].astype('datetime64[ns]')
sp = sp_all

In [46]:
sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23958 entries, 0 to 23957
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          23958 non-null  datetime64[ns]
 1   Open          23958 non-null  float64       
 2   High          23958 non-null  float64       
 3   Low           23958 non-null  float64       
 4   Close         23958 non-null  float64       
 5   Volume        23958 non-null  int64         
 6   Dividends     23958 non-null  float64       
 7   Stock Splits  23958 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 1.5 MB


In [47]:
#Drop columns which are not needed, 'Volume', 'Dividends', 'Stock Splits'
#Sort dataframe
sp = sp.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1).sort_values(by='Date',ascending=True) 
#Round to 2 decimal places
sp = sp.round(2)
sp

,Date,Open,High,Low,Close
0,1927-12-30,17.66,17.66,17.66,17.66
1,1928-01-03,17.76,17.76,17.76,17.76
2,1928-01-04,17.72,17.72,17.72,17.72
3,1928-01-05,17.55,17.55,17.55,17.55
4,1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77


# 2 - Inflation dataframe
## 2.1 Cleanup

In [33]:
inflation

,Year,Month,Value
0,1914,Jan,2.0
1,1915,Jan,1.0
2,1916,Jan,3.0
3,1917,Jan,12.5
4,1918,Jan,19.7
...,...,...,...
1314,2018,Dec,1.9
1315,2019,Dec,2.3
1316,2020,Dec,1.4
1317,2021,Dec,7.0


In [55]:
from datetime import datetime
#Drop Average column
inflation = inflation.drop(['Ave'], axis=1)
inflation = inflation.melt(id_vars='Year', var_name='Month', value_name='Value')
inflation = inflation.dropna()

# # Map month abbreviations to numerical representation
inflation['Month'] = inflation['Month'].apply(lambda x: datetime.strptime(x, '%b').month)

# # Concatenate Year and Month columns
inflation['Year'] = inflation['Year'].astype(str) + '-' + inflation['Month'].astype(str)
inflation= inflation.drop(['Month'], axis=1)
inflation['Year'] = pd.to_datetime(inflation['Year'], utc=True).dt.date
inflation = inflation.sort_values(by=['Year'], ascending=True)

inflation = inflation.rename(columns={"Year": "Date"})
inflation['Date'] = pd.to_datetime(inflation['Date'], utc=True).dt.date
inflation['Date'] = inflation['Date'].astype('datetime64[ns]')
inflation

,Date,Value
0,1914-01-01,2.0
110,1914-02-01,1.0
220,1914-03-01,1.0
330,1914-04-01,0.0
440,1914-05-01,2.1
...,...,...
1318,2022-12-01,6.5
109,2023-01-01,6.4
219,2023-02-01,6.0
329,2023-03-01,5.0


In [ ]:
inflation.sort_values(by=['Date'], ascending=True)

In [56]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1312 entries, 0 to 439
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1312 non-null   datetime64[ns]
 1   Value   1312 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 30.8 KB


# Unemployment

## Cleanup

In [34]:
unemployment = pd.read_csv("../data/unemployment_bls.csv")

In [43]:
unemployment

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2019,4.0,3.8,3.8,3.6,3.7,3.6,3.7,3.7,3.5,3.6,3.6,3.6
72,2020,3.5,3.5,4.4,14.7,13.2,11.0,10.2,8.4,7.9,6.9,6.7,6.7
73,2021,6.3,6.2,6.1,6.1,5.8,5.9,5.4,5.2,4.8,4.5,4.2,3.9
74,2022,4.0,3.8,3.6,3.6,3.6,3.6,3.5,3.7,3.5,3.7,3.6,3.5


In [36]:
# from datetime import datetime
# unemployment['Date'] = pd.to_datetime(unemployment['Date'], utc=True).dt.date
# unemployment['Date'] = unemployment['Date'].astype('datetime64[ns]')

KeyError: 'Date'

In [44]:


unemployment = unemployment.melt(id_vars='Year', var_name='Month', value_name='Value')
unemployment = unemployment.dropna()

# # Map month abbreviations to numerical representation
unemployment['Month'] = unemployment['Month'].apply(lambda x: datetime.strptime(x, '%b').month)

# # # Concatenate Year and Month columns
unemployment['Year'] = unemployment['Year'].astype(str) + '-' + unemployment['Month'].astype(str)
unemployment= unemployment.drop(['Month'], axis=1)
unemployment = unemployment.sort_values(by=['Year'], ascending=True)
unemployment = unemployment.rename(columns={"Year": "Date"})
unemployment['Date'] = pd.to_datetime(unemployment['Date'], utc=True).dt.date
unemployment= unemployment.sort_values(by=['Date'], ascending=True)
unemployment


,Date,Value
0,1948-01-01,3.4
76,1948-02-01,3.8
152,1948-03-01,4.0
228,1948-04-01,3.9
304,1948-05-01,3.5
...,...,...
910,2022-12-01,3.5
75,2023-01-01,3.4
151,2023-02-01,3.6
227,2023-03-01,3.5


# Non Farm Payroll


## Cleanup

In [ ]:

nan_count = npayroll.isna().sum() 
nan_count

In [ ]:
# Rename the columns
npayroll = npayroll.rename(columns={"DATE": "Date", "PAYEMS" :"non_farm_payroll"})
#Covert Date column from  object to DateTime
npayroll['Date'] = pd.to_datetime(npayroll['Date'], utc=True).dt.date

npayroll= npayroll.sort_values(by=['Date'], ascending=True)

# VIX

## Cleanup

In [ ]:

#Delete unwanted columnsLow
vix = vix.drop(['Open','High','Low','Adj Close','Volume'], axis=1)
#Covert Date column from  object to DateTime
vix['Date'] = pd.to_datetime(vix['Date'], utc=True).dt.date

vix= vix.sort_values(by=['Date'], ascending=True)
vix

In [ ]:
inflation


In [ ]:
cpi


In [ ]:
intRate 


In [ ]:
unemployment 


In [ ]:
npayroll 


In [ ]:
vix 

In [13]:
sp

,Date,Open,High,Low,Close
0,1927-12-30,17.66,17.66,17.66,17.66
1,1928-01-03,17.76,17.76,17.76,17.76
2,1928-01-04,17.72,17.72,17.72,17.72
3,1928-01-05,17.55,17.55,17.55,17.55
4,1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77


In [14]:
sp.index = sp.pop('Date')
sp

,Open,High,Low,Close
Date,,,,
1927-12-30,17.66,17.66,17.66,17.66
1928-01-03,17.76,17.76,17.76,17.76
1928-01-04,17.72,17.72,17.72,17.72
1928-01-05,17.55,17.55,17.55,17.55
1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...
2023-05-12,4138.54,4143.74,4099.12,4124.08
2023-05-15,4126.65,4141.25,4110.27,4136.28
2023-05-16,4127.95,4135.54,4109.86,4109.90


In [18]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

datetime.datetime(1986, 3, 19, 0, 0)

In [20]:
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=5):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date
  
  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)
    
    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))
    
    if last_time:
      break
    
    target_date = next_date

    if target_date == last_date:
      last_time = True
    
  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates
  
  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]
  
  ret_df['Target'] = Y

  return ret_df



In [21]:
# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(sp, 
                                '1990-01-01', 
                                '2023-05-18', 
                                n=5)
windowed_df

,Target Date,Target-5,Target-4,Target-3,Target-2,Target-1,Target
0,1990-01-01,344.78,347.42,346.81,348.81,350.67,353.40
1,1990-01-03,346.81,348.81,350.67,353.40,359.69,358.76
2,1990-01-04,348.81,350.67,353.40,359.69,358.76,355.67
3,1990-01-05,350.67,353.40,359.69,358.76,355.67,352.20
4,1990-01-08,353.40,359.69,358.76,355.67,352.20,353.79
...,...,...,...,...,...,...,...
8404,2023-05-12,4136.25,4138.12,4119.17,4137.64,4130.62,4124.08
8405,2023-05-15,4138.12,4119.17,4137.64,4130.62,4124.08,4136.28
8406,2023-05-16,4119.17,4137.64,4130.62,4124.08,4136.28,4109.90
8407,2023-05-17,4137.64,4130.62,4124.08,4136.28,4109.90,4158.77


In [ ]:
sp.info()

In [ ]:
sp['Date'] = sp['Date'].apply(str_to_datetime)
sp['Date']

In [ ]:
sp

In [ ]:
sp_all = pd.read_csv('../data/sp500.csv', low_memory=False,parse_dates=['Date'])

In [ ]:
sp = sp_all

In [ ]:
sp['Date'] = sp['Date'].apply(str_to_datetime)
sp['Date']

In [ ]:
sp.info()

In [ ]:
sp['Date'] = pd.to_datetime(sp['Date'], utc=True).dt.date
sp

In [ ]:
sp['Date'] = sp['Date'].astype('datetime64[ns]')

In [22]:
sp

,Open,High,Low,Close
Date,,,,
1927-12-30,17.66,17.66,17.66,17.66
1928-01-03,17.76,17.76,17.76,17.76
1928-01-04,17.72,17.72,17.72,17.72
1928-01-05,17.55,17.55,17.55,17.55
1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...
2023-05-12,4138.54,4143.74,4099.12,4124.08
2023-05-15,4126.65,4141.25,4110.27,4136.28
2023-05-16,4127.95,4135.54,4109.86,4109.90


In [60]:
#left_df.merge(right_df, on='user_id', how='left')
spf= sp.merge(inflation, on='Date', how='left').sort_values(by=['Date'], ascending=True)

In [58]:
inflation

,Date,Value
0,1914-01-01,2.0
110,1914-02-01,1.0
220,1914-03-01,1.0
330,1914-04-01,0.0
440,1914-05-01,2.1
...,...,...
1318,2022-12-01,6.5
109,2023-01-01,6.4
219,2023-02-01,6.0
329,2023-03-01,5.0


In [63]:
spf[spf['Date'] > '1928-01-31']

,Date,Open,High,Low,Close,Value
22,1928-02-01,17.53,17.53,17.53,17.53,-1.7
23,1928-02-02,17.63,17.63,17.63,17.63,NaN
24,1928-02-03,17.40,17.40,17.40,17.40,NaN
25,1928-02-06,17.45,17.45,17.45,17.45,NaN
26,1928-02-07,17.44,17.44,17.44,17.44,NaN
...,...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08,NaN
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28,NaN
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90,NaN
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77,NaN


In [64]:
spf['Value'] = spf['Value'].fillna(method='ffill')
spf

,Date,Open,High,Low,Close,Value
0,1927-12-30,17.66,17.66,17.66,17.66,NaN
1,1928-01-03,17.76,17.76,17.76,17.76,NaN
2,1928-01-04,17.72,17.72,17.72,17.72,NaN
3,1928-01-05,17.55,17.55,17.55,17.55,NaN
4,1928-01-06,17.66,17.66,17.66,17.66,NaN
...,...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08,5.0
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28,5.0
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90,5.0
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77,5.0


In [67]:
spf[spf['Date'] > '1989-12-31']

,Date,Open,High,Low,Close,Value
15549,1990-01-02,353.40,359.69,351.98,359.69,4.6
15550,1990-01-03,359.69,360.59,357.89,358.76,4.6
15551,1990-01-04,358.76,358.76,352.89,355.67,4.6
15552,1990-01-05,355.67,355.67,351.35,352.20,4.6
15553,1990-01-08,352.20,354.24,350.54,353.79,4.6
...,...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08,5.0
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28,5.0
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90,5.0
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77,5.0


In [66]:
spf.to_csv('spf.csv')

In [68]:
inflation

,Date,Value
0,1914-01-01,2.0
110,1914-02-01,1.0
220,1914-03-01,1.0
330,1914-04-01,0.0
440,1914-05-01,2.1
...,...,...
1318,2022-12-01,6.5
109,2023-01-01,6.4
219,2023-02-01,6.0
329,2023-03-01,5.0


In [69]:
sp

,Date,Open,High,Low,Close
0,1927-12-30,17.66,17.66,17.66,17.66
1,1928-01-03,17.76,17.76,17.76,17.76
2,1928-01-04,17.72,17.72,17.72,17.72
3,1928-01-05,17.55,17.55,17.55,17.55
4,1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77


In [70]:
cpi = pd.read_csv("../data/cpi_alpha_cleaned-final.csv")
cpi

,Date,CPI
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,303.363
23954,2023-05-15,303.363
23955,2023-05-16,303.363
23956,2023-05-17,303.363
